# ema黃金交叉進場, 死亡交叉出場
# ema黃金交叉進場, sma抵扣價出場

### vectorbt
[官方文件](https://vectorbt.dev/)

### 參考資料
1. [入門量化交易 – 資料/指標/模型/回測/交易全套分享](https://weikaiwei.com/finance/%E5%85%A5%E9%96%80%E9%87%8F%E5%8C%96%E4%BA%A4%E6%98%93-%E8%B3%87%E6%96%99-%E6%8C%87%E6%A8%99-%E6%A8%A1%E5%9E%8B-%E5%9B%9E%E6%B8%AC-%E4%BA%A4%E6%98%93%E5%85%A8%E5%A5%97%E5%88%86%E4%BA%AB/)
2. [Algo Trad Pipeline](https://voidful.dev/jupyter/2021/02/20/cryptotaipynb.html)

In [ ]:
try:
    import vectorbt
except ModuleNotFoundError as e:
    !pip install vectorbt    

try:
    import yfinance
except ModuleNotFoundError as e:
    !pip install yfinance   

try:
    import ccxt
except ModuleNotFoundError as e:
    !pip install ccxt

try:
    import talib
except ModuleNotFoundError as e:
    !pip install talib_binary         


from IPython.display import clear_output
clear_output()

!exit

In [ ]:
#官方範例
import vectorbt as vbt
price = vbt.YFData.download('BTC-USD').get('Close')
pf = vbt.Portfolio.from_holding(price, init_cash=100)
pf.total_profit()


4280.53097971089

In [ ]:
#官方範例
fast_ma = vbt.MA.run(price, 10)
slow_ma = vbt.MA.run(price, 50)
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf = vbt.Portfolio.from_signals(price, entries, exits, init_cash=100)
pf.total_profit()
pf.stats()

### vectorbt內建可以接CCXT抓資料,但又包了一層,干擾原本CCXT的語法.

In [ ]:
ccxt_data = vbt.CCXTData.download(
...     "BTC/USDT",
...     start='1 month ago UTC',
...     end='now UTC',
...     timeframe='1d')

ccxt_df = ccxt_data.get()
ccxt_df.tail(3)

In [ ]:
#@title 回測設定
symbol_name = "SOL/USD" #@param {type:"string"}
timeframe = "5m" #@param ['15s', '1m', '5m', '15m', '1h', '4h', '1d', '3d', '1w', '2w', '1M']
kline_number = 5000 #@param {type:"slider", min:100, max:5000, step:100}
ema_s = 10 #@param {type:"integer"}
ema_m = 50 #@param {type:"integer"}
sma_deduct = 20 #@param {type:"integer"}
init_cash = 100 #@param {type:"integer"}
#FTX上限5000根K棒...

import ccxt
import pandas as pd

exchange = ccxt.ftx()
columns = ["Time","Open","High","Low","Close","Volume"]
ohlc_df = pd.DataFrame(exchange.fetch_ohlcv(symbol_name,timeframe=timeframe,limit=kline_number))
ohlc_df.columns = columns

ohlc_df["Time"] = pd.to_datetime(ohlc_df["Time"],unit='ms')
ohlc_df.set_index("Time",inplace=True)
ohlc_df.tail(3)


In [ ]:
import talib

ohlc_df["ema_sline"] = talib.EMA(ohlc_df["Close"],timeperiod=ema_s)
ohlc_df["ema_m1line"] = talib.EMA(ohlc_df["Close"],timeperiod=ema_m)

ema_entry = [False]
for i in range(1,ohlc_df.shape[0]):
    
    if ohlc_df["ema_sline"].iat[i] > ohlc_df["ema_m1line"].iat[i] and ohlc_df["ema_sline"].iat[i-1] < ohlc_df["ema_m1line"].iat[i-1]:
        ema_entry.append(True)
    else:
        ema_entry.append(False) 

ema_exit = [False]
for i in range(1,ohlc_df.shape[0]):
    
    if ohlc_df["ema_sline"].iat[i] < ohlc_df["ema_m1line"].iat[i] and ohlc_df["ema_sline"].iat[i-1] > ohlc_df["ema_m1line"].iat[i-1]:
        ema_exit.append(True)
    else:
        ema_exit.append(False) 

sma_d_exit = [False]
for i in range(1,ohlc_df.shape[0]):
    if ohlc_df['Close'].iat[i] < ohlc_df['Close'].iat[i-sma_deduct]:
        sma_d_exit.append(True)
    else:
        sma_d_exit.append(False)    


ohlc_df["ema_entry"] = ema_entry
ohlc_df["ema_exit"] = ema_exit
ohlc_df["sma_d_exit"] = sma_d_exit


ema_pf = vbt.Portfolio.from_signals(close=ohlc_df["Close"], entries=ohlc_df["ema_entry"], exits=ohlc_df["ema_exit"], init_cash=init_cash)
sma_d_pf = vbt.Portfolio.from_signals(close=ohlc_df["Close"], entries=ohlc_df["ema_entry"], exits=ohlc_df["sma_d_exit"], init_cash=init_cash)
#pf.stats()
ohlc_df.tail(3)

In [ ]:
ema_pf_orders = ema_pf.orders.records_readable
sma_d_pf_orders = sma_d_pf.orders.records_readable

ema_pf_orders.set_index("Timestamp", inplace=True)
sma_d_pf_orders.set_index("Timestamp", inplace=True)

ema_pf_orders.drop(columns=["Order Id","Column","Size","Price","Fees"],inplace=True)
sma_d_pf_orders.drop(columns=["Order Id","Column","Size","Price","Fees"],inplace=True)

ema_ohlc_df = pd.merge(ohlc_df,ema_pf_orders,right_index=True,left_index=True,how="outer")
sma_d_ohlc_df = pd.merge(ohlc_df,sma_d_pf_orders,right_index=True,left_index=True,how="outer")




In [ ]:
import plotly.graph_objects as px
from plotly.subplots import make_subplots

#畫K線
def plotly_kline(data):
    candlestick = px.Candlestick(x=data.index,
                    open=data['Open'],
                    high=data['High'],
                    low=data['Low'],
                    close=data['Close'],name="Kline")

    ema_sline = px.Scatter(x=data.index, 
                    y=data['ema_sline'],
                    line=dict(color='orange', width=1),
                    mode='lines',name=f"EMA{ema_s}")
    
    ema_m1line = px.Scatter(x=data.index, 
                    y=data['ema_m1line'],
                    line=dict(color='green', width=1),
                    mode='lines',name=f"EMA{ema_m}")
    
    buy_side = data["Side"].isin(["Buy"])
    sell_side = data["Side"].isin(["Sell"])
    
    buy_side_markers = px.Scatter(x=data[buy_side].index,
                    y=data[buy_side]["Low"]*0.95, mode="markers", name="Buy",
                    marker=(dict(symbol="arrow-up",size=10,color='green')))
    
    sell_side_markers = px.Scatter(x=data[sell_side].index,
                    y=data[sell_side]["High"]*1.05, mode="markers", name="Sell",
                    marker=(dict(symbol="arrow-down",size=10,color='red')))
    
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True, row_heights=[1],vertical_spacing=0.01)
    fig.add_traces([candlestick, ema_sline,ema_m1line, sell_side_markers, buy_side_markers],rows=1,cols=1)
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig["layout"]["template"]="plotly_dark"
    fig.show()    

In [ ]:
#ema黃金交叉進場，死亡交叉出場
ema_pf.stats()

In [ ]:
#ema黃金交叉進場，收盤低於sma扺扣價出場
sma_d_pf.stats()

In [ ]:
#畫K線,標出買賣點
plotly_kline(ema_ohlc_df)

In [ ]:
#畫K線,標出買賣點
plotly_kline(sma_d_ohlc_df)

In [ ]:
ema_pf.plot().show()

In [ ]:
sma_d_pf.plot().show()